In [3]:
import pyprind
import pandas as pd
import os
import sys
basepath = 'aclImdb_v1\\aclImdb'

In [4]:
os.chdir(r'C:\Users\Bclea\Downloads')

In [5]:
#data wrangling
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s ,l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            new_row = pd.DataFrame({'review': [txt], 'sentiment': [labels[l]]})
            df = pd.concat([df, new_row], ignore_index=True)
            #df = pd.concat([df, pd.DataFrame([txt, labels[l]])], ignore_index=True)
            pbar.update()


In [6]:
df = df.sample(frac=1).reset_index(drop=True)


In [7]:
df.tail(20)

,review,sentiment
49980,I had to watch this one for my Canadian cinema...,0
49981,All this dismaying waste of film stock needs i...,0
49982,"This movie makes ""Glitter"" look like ""Schindle...",0
49983,"About five years ago, my friend and I went to ...",0
49984,Not a `woman film' but film for the gang. One ...,0
49985,Why couldn't the end of the movie have been Se...,1
49986,The world is made up two different types of mo...,1
49987,After apprehending the man responsible for the...,0
49988,Kramer vs. Kramer is the story of a marital br...,1
49989,This four-hour miniseries production is about ...,0


In [8]:
df.shape

(50000, 2)

In [9]:
#data cleaning
import re
def preprocesser(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text= (re.sub('[\W]+', ' ', text.lower())+ ' '.join(emoticons).replace('-', ''))
    return text

In [10]:
df['review'] = df['review'].apply(preprocesser)

In [11]:
#bag-of-words-model
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = df['review'].to_numpy()

In [12]:
bag = count.fit_transform(docs)

In [13]:
print(count.vocabulary_)

{'it': 47965, 'strange': 88069, 'while': 101092, 'the': 91976, 'film': 33967, 'features': 33335, 'full': 36407, 'rated': 74574, 'sex': 82017, 'scenes': 80322, 'and': 4779, 'violent': 99212, 'murders': 61976, 'never': 63338, 'feels': 33400, 'as': 6476, 'shocking': 82889, 'ought': 66247, 'to': 93189, 'group': 40011, 'of': 65112, 'scientists': 80692, 'go': 38629, 'an': 4651, 'island': 47885, 'in': 45768, 'caribbean': 15199, 'investigate': 47531, 'radioactive': 74027, 'incident': 45902, 'upon': 97698, 'their': 92050, 'arrival': 6299, 'mutated': 62137, 'islander': 47887, 'goes': 38752, 'about': 2024, 'happy': 41354, 'business': 14040, 'murdering': 61973, 'men': 58679, 'having': 41833, 'his': 43157, 'way': 100441, 'with': 101882, 'women': 102087, 'doesn': 27022, 'always': 4267, 'seem': 81351, 'work': 102274, 'out': 66279, 'that': 91940, 'among': 4552, 'sored': 85688, 'acts': 2547, 'we': 100479, 'find': 34120, 'some': 85469, 'lesbian': 53470, 'encounters': 30183, 'three': 92531, 'male': 56335

In [14]:
print(bag.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [15]:
#assessing word relevancy via term frequency inverse document frequency
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
#processing documents into tokens
def tokenizer(text):
    return text.split()

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    if isinstance(text, np.ndarray):
        text = np.array2string(text)
        return [porter.stem(word) for word in text.split()]
    else:
        return [porter.stem(word) for word in text.split()]
tokenizer_porter(docs)

["['it",
 's',
 'strang',
 'while',
 'the',
 'film',
 'featur',
 'full',
 'x',
 'rate',
 'sex',
 'scene',
 'and',
 'violent',
 'murder',
 'it',
 'never',
 'feel',
 'as',
 'shock',
 'as',
 'it',
 'ought',
 'to',
 'a',
 'group',
 'of',
 'scientist',
 'go',
 'to',
 'an',
 'island',
 'in',
 'the',
 'caribbean',
 'to',
 'investig',
 'a',
 'radioact',
 'incid',
 'upon',
 'their',
 'arriv',
 'a',
 'mutat',
 'island',
 'goe',
 'about',
 'the',
 'happi',
 'busi',
 'of',
 'murder',
 'the',
 'men',
 'and',
 'have',
 'hi',
 'way',
 'with',
 'the',
 'women',
 'doesn',
 't',
 'it',
 'alway',
 'seem',
 'to',
 'work',
 'out',
 'that',
 'way',
 'among',
 'the',
 'sore',
 'act',
 'we',
 'find',
 'a',
 'some',
 'lesbian',
 'encount',
 'a',
 'three',
 'way',
 'with',
 'male',
 'prostitut',
 'assort',
 'heterosexu',
 'coupl',
 'and',
 'the',
 'rape',
 'of',
 'an',
 'alreadi',
 'dead',
 'bodi',
 'even',
 'though',
 'it',
 's',
 'all',
 'fulli',
 'explicit',
 'it',
 'fail',
 'to',
 'ever',
 'shock',
 'or',
 

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter(docs) if w not in stop]



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bclea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


["['it",
 'strang',
 'film',
 'featur',
 'full',
 'x',
 'rate',
 'sex',
 'scene',
 'violent',
 'murder',
 'never',
 'feel',
 'shock',
 'ought',
 'group',
 'scientist',
 'go',
 'island',
 'caribbean',
 'investig',
 'radioact',
 'incid',
 'upon',
 'arriv',
 'mutat',
 'island',
 'goe',
 'happi',
 'busi',
 'murder',
 'men',
 'hi',
 'way',
 'women',
 'alway',
 'seem',
 'work',
 'way',
 'among',
 'sore',
 'act',
 'find',
 'lesbian',
 'encount',
 'three',
 'way',
 'male',
 'prostitut',
 'assort',
 'heterosexu',
 'coupl',
 'rape',
 'alreadi',
 'dead',
 'bodi',
 'even',
 'though',
 'fulli',
 'explicit',
 'fail',
 'ever',
 'shock',
 'stir',
 'meant',
 'soon',
 'sex',
 'goe',
 'fulli',
 'pornograph',
 'lose',
 'edg',
 'suspens',
 'disbelief',
 'broken',
 'realiz',
 'watch',
 'peopl',
 'sex',
 'blood',
 'gore',
 'murder',
 'given',
 'thi',
 'amato',
 'flick',
 'realli',
 'tame',
 'much',
 'round',
 'experi',
 'watch',
 'similar',
 'erot',
 'night',
 'live',
 'dead',
 '2',
 "10'",
 "'thi",
 'movi',

In [18]:
#document classification
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Define TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

# Corrected small_param_grid with double underscores for accessing pipeline parameters
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0],
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0],
    }
]

# Create pipeline
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])

# Perform grid search with the corrected parameter grid
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)

# Assuming X_train and y_train are already defined
gs_lr_tfidf.fit(X_train, y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\Bclea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x000001F0D5314CC0>; total time=   3.5s


C:\Users\Bclea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x000001F0D5314CC0>; total time=   3.2s


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x000001F0D5314CC0>, vect__use_idf=False; total time=  11.8s


C:\Users\Bclea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x000001F0D5314CC0>, vect__use_idf=False; total time=  11.3s


C:\Users\Bclea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x000001F0D5314CC0>,
                                              <function tokenizer_porter at 0x000001F0D...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x000001F0D5314CC0>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)